# Welcome to **TrumpSpeak:**

A text-to-speech (TTS) application where you can generate a voice that sounds like the dumbest idiot to have ever been born. The model was trained using Tacotron and the voice is synthesized using a pretrained WaveRNN vocoder. The Trump model already sounded great at 30K steps (from SCRATCH, not a refinement of existing models - crazy, right?!) using 8 hours of professionally cleaned and sliced audio. Overfitting then caused a massive increase in duration loss. I generated the audio and transcript dataset with a tool I specifically created for this specific purpose. Preparing the data only took about 4 hours, not including the time to rip the videos from YouTube. Training took 8 hours. I have no intention of releasing my dataset preparation tool due to how  insanely trivial it makes the process of duplicating someone's voice. Most of the code used here was taken from [ForwardTacotron](https://github.com/as-ideas/ForwardTacotron).

Have fun with it. I can't wait to see a bunch of offensive content made with it.

Sincerely,

**Justsome Fuckingasshole**



1.   Click Setup. Give it like 2 minutes to figure all its bullshit out.
2.   Wait for the circle thing to stop spinning.
3. Click the Clear Output button just below the Start button to clear the shit that just piled up.
4. Type some stupid bullshit in the "speech" box in the section labeled "Make Trump Say Shit"
5. You can leave the other settings alone.
6. Press the Play Button and wait for it to process
7. An audio player will appear once it's ready
8. If the voice is fucked, try changing the model iteration to another number. Lower number = more natural pacing but sounds robotic. Higher number = voice forgets how to say shit correctly.
9. You can automate speech generation by editing the "speech.txt" file. Each line produces a new file.





In [ ]:
#@title Setup

#generate a speech text file which can be modified and used to automate TTS generation
speech_file = open("speech.txt","a")
speech_file.write("The Chinese corona virus is turning our children into gay communist vegans and that's a terrible thing to have happen to such a great and powerful country.\nWe must put an end to this attack on our nation by the democrats which I call corona gate.\nSo please come to my rallies and be sure not to wear those stupid masks.")
speech_file.close()

!git clone https://github.com/gormulka/TrumpSpeak.git
!mkdir /content/output
%cd /content/TrumpSpeak/
!apt-get install -y espeak
!pip install -r requirements.txt
!pip install pydub
!pip install librosa==0.7.2

from utils import hparams as hp
import IPython.display as ipd
from pydub import AudioSegment

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

import os
import IPython
from IPython.display import Audio as play
from pathlib import Path
import time

player = []

def playAllAudio():
  %cd /content/output/
  out_path = "/content/output/"
  files = os.listdir(out_path)
  latest_file = max(files, key=os.path.getctime)
  player = [play(out_path + files[0])]
  playerIndex = 0
  print()
  for x in range(0,len(files)):
    if Path(out_path + files[x]).suffix == '.wav':
      print(files[x])
      player.append(play(out_path + files[x]))
      playerIndex+=1
      time.sleep(1)
      IPython.display.display(player[playerIndex])
      time.sleep(1)
      print()

def playMostRecent():
  %cd /content/output/
  out_path = "/content/output/"
  files = os.listdir(out_path)
  latest_file = max(files, key=os.path.getctime)
  print(latest_file)
  #player = [play(latest_file)]
  player.append(play(out_path + latest_file))
  IPython.display.display(player[len(player)-1])
  time.sleep(1)


print('\n\n')

print("Done! I'm ready to make Trump say some stupid shit.")

In [2]:
import os
from IPython.display import Audio

def playMostRecent():
    out_path = "/content/output/"
    files = [os.path.join(out_path, f) for f in os.listdir(out_path) if os.path.isfile(os.path.join(out_path, f))]
    if files:
        latest_file = max(files, key=os.path.getctime)
        print(f"Playing file: {latest_file}")
        display(Audio(latest_file))
    else:
        print("No files found in the output directory.")

!pip install librosa==0.8.1 numba==0.48 torch numpy scipy matplotlib

# Ensure output directory exists
output_path = "/content/output/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Set parameters
speech = "The Chinese corona virus is turning our children into gay communist vegans"
model_iteration = 80
talking_speed = 0.9
modelWeights = f"{model_iteration}K.pyt"

# Run gen_forward.py
!python /content/TrumpSpeak/gen_forward.py \
  --alpha $talking_speed \
  --input_text "$speech" \
  --hp_file "/content/TrumpSpeak/pretrained/pretrained_hparams.py" \
  --tts_weights "/content/TrumpSpeak/checkpoints/ljspeech_tts.forward/$modelWeights" \
  wavernn \
  --voc_weights "/content/TrumpSpeak/pretrained/wave_800K.pyt" \
  --batched --target=4096 --overlap=32

# Play the most recent output
playMostRecent()


/content/TrumpSpeak
Traceback (most recent call last):
  File "/content/TrumpSpeak/gen_forward.py", line 3, in <module>
    from models.fatchord_version import WaveRNN
  File "/content/TrumpSpeak/models/fatchord_version.py", line 6, in <module>
    from utils.dsp import *
  File "/content/TrumpSpeak/utils/dsp.py", line 4, in <module>
    import librosa
  File "/usr/local/lib/python3.10/dist-packages/librosa/__init__.py", line 12, in <module>
    from . import core
  File "/usr/local/lib/python3.10/dist-packages/librosa/core/__init__.py", line 125, in <module>
    from .time_frequency import *  # pylint: disable=wildcard-import
  File "/usr/local/lib/python3.10/dist-packages/librosa/core/time_frequency.py", line 11, in <module>
    from ..util.exceptions import ParameterError
  File "/usr/local/lib/python3.10/dist-packages/librosa/util/__init__.py", line 77, in <module>
    from .utils import *  # pylint: disable=wildcard-import
  File "/usr/local/lib/python3.10/dist-packages/librosa/ut

ValueError: max() arg is an empty sequence

In [ ]:
#@title Process entire speech file (speech.txt)
import os
import IPython
from IPython.display import Audio as play
from pathlib import Path
import time

lineIndex = 0
playerIndex = 0
player = []
model_iteration = 80 #@param {type:"slider", min:40, max:100, step:20}
talking_speed = 0.9 #@param {type:"slider", min:0.5, max:1, step:0.01}
modelWeights = str(model_iteration) + "K.pyt"

out_path = "/content/output/"

speechPath = '/content/speech.txt'

files = os.listdir(out_path)
for file in files:
  file = out_path + file
with open(speechPath) as fp:
  line = fp.readline()
  cnt = 1
  lineIndex = 0
  playerIndex = 0
  player = []
  while line:
    print("Line {}: {}".format(cnt, line.strip()))

    cnt += 1
    %cd /content/TrumpSpeak//
    !python /content/TrumpSpeak/gen_forward.py --alpha 0.9 --input_text "$line" --hp_file "/content/TrumpSpeak/pretrained/pretrained_hparams.py" --tts_weights "/content/TrumpSpeak/checkpoints/ljspeech_tts.forward/$modelWeights" wavernn --voc_weights "/content/TrumpSpeak/pretrained/wave_800K.pyt" --batched --target=1024 --overlap=128
    playMostRecent()
    line = fp.readline()


#Not particularly useful stuff

In [ ]:
#@title Print Available Checkpoints
%cd /content/TrumpSpeak//checkpoints/ljspeech_tts.forward/
%ls
%cd /content/TrumpSpeak/checkpoints/ljspeech_tts.tacotron/
%ls

In [ ]:
#@title Play All Audio Files
import os
import IPython
from IPython.display import Audio as play
from pathlib import Path
import time

def playAllAudio():
  out_path = "/content/output/"
  %cd $out_path
  files = os.listdir(out_path)
  latest_file = max(files, key=os.path.getctime)
  player = [play(out_path + files[0])]
  playerIndex = 0
  print()
  for x in range(0,len(files)):
    if Path(out_path + files[x]).suffix == '.wav':
      print(files[x])
      player.append(play(out_path + files[x]))
      playerIndex+=1
      time.sleep(1)
      IPython.display.display(player[playerIndex])
      time.sleep(1)
      print()


playAllAudio()
